<a href="https://colab.research.google.com/github/springboardmentor0327/Text_Summarization_Infosys_Internship_Oct2024/blob/Sameer/freq_method_with_rouge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=56b7256551d818b3fd4e2d3b21aea7957a2a74c757b1f0628571e15683e81e63
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from rouge_score import rouge_scorer
import pandas as pd

In [ ]:
# Download required resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')  # For lemmatization

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
text='''

Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding.

Time Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. "Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility," chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.

TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.


'''

In [ ]:
def solve(text, max_words=30):
    # Load English stopwords
    stopwords1 = set(stopwords.words("english"))

    # Tokenize text into words
    words = word_tokenize(text)

    # Create a frequency table for words (ignoring stopwords)
    freqTable = {word.lower(): words.count(word.lower()) for word in words if word.lower() not in stopwords1}

    # Tokenize text into sentences
    sentences = sent_tokenize(text)

    # Create a sentence value dictionary and sentence score
    sentenceValue = {}
    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                sentenceValue[sentence] = sentenceValue.get(sentence, 0) + freq

    # Calculate the sum of sentence values
    sumValues = sum(sentenceValue.values())

    # Calculate the average sentence value
    average = int(sumValues / len(sentenceValue))

    # Generate the summary with max and min word constraints
    summary_sentences = []
    current_word_count = 0
    min_words = max_words - 10  # Default minimum words limit

    for sentence in sentences:
        if sentenceValue.get(sentence, 0) > (1.2 * average):
            sentence_word_count = len(word_tokenize(sentence))
            if min_words <= current_word_count + sentence_word_count <= max_words:
                summary_sentences.append(sentence)
                current_word_count += sentence_word_count

    summary = ' '.join(summary_sentences)
    return summary

# Calculate and print word count before and after summarization
original_word_count = len(word_tokenize(text))
summary_text = solve(text,max_words=40)
summary_word_count = len(word_tokenize(summary_text))

print('Original Text:')
print(text)
print(f"\nWord Count (Original): {original_word_count}\n")

print("Summary of Text:\n")
print(summary_text)
print(f"\nWord Count (Summary): {summary_word_count}")


Original Text:


Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will t

**ROUGE** Module


In [ ]:
def solve(text, threshold=1.2):
    # Initialize stopwords, lemmatizer, and tokenizer
    stopwords_set = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()

    # Tokenize text into words
    words = word_tokenize(text)

    # Lemmatize words and filter out stopwords
    lemmatized_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stopwords_set and word.isalnum()]

    # Create a frequency table for lemmatized words
    freq_table = {word: lemmatized_words.count(word) for word in lemmatized_words}

    # Tokenize text into sentences
    sentences = sent_tokenize(text)

    # Create a sentence value dictionary to score sentences
    sentence_value = {}
    for sentence in sentences:
        for word, freq in freq_table.items():
            if word in sentence.lower():
                sentence_value[sentence] = sentence_value.get(sentence, 0) + freq

    # Calculate the sum of sentence values
    sum_values = sum(sentence_value.values())

    # Calculate the average sentence value
    average = int(sum_values / len(sentence_value)) if len(sentence_value) > 0 else 0

    # Generate the summary based on the threshold
    summary = ' '.join([sentence for sentence in sentences if sentence_value.get(sentence, 0) > (threshold * average)])

    return summary

# Function to compute ROUGE scores
def compute_rouge_scores(reference, summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, summary)
    return scores

# Function to evaluate the model on the text with ROUGE metrics
def evaluate_rouge(text, reference_summary, model_function, threshold=1.2):
    generated_summary = model_function(text, threshold)
    rouge_scores = compute_rouge_scores(reference_summary, generated_summary)

    # Display ROUGE scores
    print(f"Generated Summary:\n{generated_summary}\n")
    print(f"ROUGE-1: {rouge_scores['rouge1'].fmeasure:.4f}")
    print(f"ROUGE-2: {rouge_scores['rouge2'].fmeasure:.4f}")
    print(f"ROUGE-L: {rouge_scores['rougeL'].fmeasure:.4f}")
    return rouge_scores

# Example usage
text = '''
Samsung was founded by Lee Byung-chul in 1938 as a trading company. Over the next three decades, the group diversified into areas including food processing, textiles, insurance, securities, and retail. Samsung entered the electronics industry in the late 1960s and the construction and shipbuilding industries in the mid-1970s; these areas would drive its subsequent growth. Following Lee's death in 1987, Samsung was separated into five business groups – Samsung Group, Shinsegae Group, CJ Group, Hansol Group, and JoongAng Group.
'''

# Assume this is the reference summary provided
reference_summary = 'Samsung was founded in 1938 as a trading company. Samsung entered electronics in the 1960s and shipbuilding in the 1970s. Following Lee\'s death, Samsung was separated into five business groups.'

# Evaluate model and print ROUGE scores
print('Original Text:')
print(text)

print("\nSummary of Text:\n")
evaluate_rouge(text, reference_summary, solve, threshold=1.2)


Original Text:

Samsung was founded by Lee Byung-chul in 1938 as a trading company. Over the next three decades, the group diversified into areas including food processing, textiles, insurance, securities, and retail. Samsung entered the electronics industry in the late 1960s and the construction and shipbuilding industries in the mid-1970s; these areas would drive its subsequent growth. Following Lee's death in 1987, Samsung was separated into five business groups – Samsung Group, Shinsegae Group, CJ Group, Hansol Group, and JoongAng Group.


Summary of Text:

Generated Summary:
Following Lee's death in 1987, Samsung was separated into five business groups – Samsung Group, Shinsegae Group, CJ Group, Hansol Group, and JoongAng Group.

ROUGE-1: 0.5091
ROUGE-2: 0.3396
ROUGE-L: 0.4000


{'rouge1': Score(precision=0.5833333333333334, recall=0.45161290322580644, fmeasure=0.509090909090909),
 'rouge2': Score(precision=0.391304347826087, recall=0.3, fmeasure=0.33962264150943394),
 'rougeL': Score(precision=0.4583333333333333, recall=0.3548387096774194, fmeasure=0.39999999999999997)}